<a href="https://colab.research.google.com/github/danchaud-vincent/tensorflow-deep-learning/blob/main/06_transfer_learning_in_tensorflow_part_3_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 06 - Transfer Learning with TensorFlow Part 3: Scaling up (Food Vision mini)

In the previous two notebooks (**transfer learning part 1: feature extraction** and **part 2: fine-tuning**) we've seen the power of transfer learning.

Now we know our smaller modelling experiments are working, it's time to step things up a notch with more data.

This is a common practice in machine learning and deep learning: get a model working on a small amount of data before scaling it up to a larger amount of data.

It's time to get closer to our Food Vision project coming to life. In this notebook we're going to scale up from  using 10 classes of the Food101 data to using all of the classes in the Food101 dataset.

Our goal is to **beat the original Food101 paper's results with 10% of data**.

![](https://raw.githubusercontent.com/danchaud-vincent/tensorflow-deep-learning/main/images/06-ml-serial-experimentation.png)
***Machine learning practitioners are serial experimenters. Start small, get a model working, see if yout experiments work then gradually scale them up to where you want to go (we're going to be looking at scaling up throughout this notebook).*

## What we're going to cover

We're going to go through the follow with TensorFlow:
- Downloading and preparing 10% of the Food101 data (10% of training data).
- Training a feature extraction transfer learning model on 10% of the Food101 training data.
- Fine-tuning our feature extraction model.
- Saving and loaded our trained model.
- Evaluating the performance of our Food Vision model trained on 10% of the training data
  - Finding our model's most wrong predictions
- Making predictions with our Food Vision model on custom images of food.

In [1]:
# Are we using a GPU?
!nvidia-smi

Thu Oct 13 20:41:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Creating Helper Functions


We've created a series of helper functions throughout the previous notebooks. Instead of rewriting them (tedious), we'll import the `helper_functions.py` file from the Github repo.


In [2]:
# Get helper functions file
!wget https://raw.githubusercontent.com/danchaud-vincent/tensorflow-deep-learning/main/utils/helper_functions.py

--2022-10-13 20:43:20--  https://raw.githubusercontent.com/danchaud-vincent/tensorflow-deep-learning/main/utils/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2631 (2.6K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   2.57K  --.-KB/s    in 0s      

2022-10-13 20:43:20 (43.4 MB/s) - ‘helper_functions.py’ saved [2631/2631]



In [3]:
# Import series of helper function for the notebook
from helper_functions import plot_loss_curves, unzip_data, walk_through_dir, create_tensorboard_callback

## 101 Food Classes: Working with less data

